# 列舉所有資產的riskiness r

In [1]:
import sympy
from pandas import Series,DataFrame
from scipy.optimize import fsolve
import math
import time
import pandas as pd
import numpy  as  np
import os

# 無風險利率
* 我們須扣除無風險利率，得到超額報酬率以計算Riskiness r
* 目前使用LIBOR [1個月利率](https://www.global-rates.com/interest-rates/libor/american-dollar/usd-libor-interest-rate-overnight.aspx)

In [2]:
rfrate = 2.45785 / 12 /100
rfrate

0.0020482083333333333

# 關於超額報酬的數量級:
* 將要帶入公式的數值調整到介在0-100間，目的是為了讓大部分的Riskiness r介在0-100間。

In [3]:
def f1(x,arr_returns):
    return sum( np.exp(-1*arr_returns/x) ) - len(arr_returns)

In [4]:
def get_riskiness_r(guess , arr_returns):
    while (guess<1000):
        risk2 = fsolve(f1,guess,arr_returns)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]

# 計算Riskiness R，並依照R值排序，傳回前十名的ETF

In [62]:
def get_top10_etf(data_file):
    # 讀檔
    #data_file = '../../HW1/ETF_data_set/return_data/Alternatives ETF List (35)/Alternatives ETF List (35)_return.csv';
    df = pd.read_csv(data_file)
    etf_list = list(df.columns)[1:]
    df = df.iloc[:,1:]
    
    # 計算每支ETF的riskiness r
    etf_dic = {}
    for etf_name in etf_list:
        # 過濾平均值正負
        #if df[etf_name].mean(axis=0) < 0:
        #    continue
        arr_returns = df[etf_name] - rfrate
        #arr_returns = arr_returns*1000      #調整數量級
        guess = 10**(-5)
        #### 列出細項
        #print('%s指標值:  %f' %(etf_name,risk2[0]) )
        etf_dic[etf_name] = get_riskiness_r(guess,arr_returns)
    
    # ETF依照riskiness r排序
    sorted_x = sorted(etf_dic.items(), key=lambda kv: kv[1])
        #sorted_reversed_x = list(reversed(sorted_x))
    
    # 取前10名
    top10 = sorted_x[:5]
        # top10 = sorted_reversed_x[:10]
    #print(top10)
    
    # 取前10的ETF名稱
    etf_top10 = [etf[0] for etf in top10]
    
    #前10的平均
    top10_average = df[etf_top10].mean(axis=1)
    
    #print(etf_top10)
    return etf_top10 , top10_average

# 讀取全部ETF資料夾

In [18]:
data_path = '../../HW1/ETF_data_set/RE/'
allgroup_top10_return = DataFrame()
for subdir in os.listdir(data_path):
    subdir_path = os.path.join(data_path, subdir)
    if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
        file = subdir + '_return.csv'
        file_path = os.path.join(subdir_path, file)
        #print(file_path)
        if os.path.isfile(file_path):
            # 提出[前10名的名稱,前10名的平均各期報酬]
            [ top10_etf,allgroup_top10_return[subdir] ] = get_top10_etf(file_path)
            print('Group:',subdir)
            print('Top10:',top10_etf)
            print('================')

Group: Alternatives ETF List (35)
Top10: ['MOM', 'RLY', 'CHEP', 'VAMO', 'FTLS']
Group: Broad Asia ETF List (44)
Top10: ['AXJL', 'IQDE', 'HEEM', 'DBAP', 'AIA']
Group: Commodity ETF List (125)
Top10: ['DBA', 'BCM', 'RJZ', 'DBB', 'UCI']
Group: Consumer Discretionary Equity ETF List (31)
Top10: ['RETL', 'FDIS', 'IYC', 'VCR', 'XLY']
Group: Consumer Staples Equity ETF List (26)
Top10: ['WBIE', 'PSL', 'PSCC', 'VDC', 'EMCG']
Group: Crude Oil ETF List (22)
Top10: ['UCO', 'DTO', 'SCO', 'USL', 'SZO']
Group: Currency ETF List (36)
Top10: ['FXCH', 'FXS', 'FXA', 'CEW', 'DBV']
Group: Developed Asia Pacific ETF List (108)
Top10: ['DOL', 'JPMV', 'DTH', 'IDOG', 'DOO']
Group: Developed Markets ETF List 1 (100)
Top10: ['GII', 'JXI', 'WDIV', 'DOL', 'GUNR']
Group: Developed Markets ETF List 2 (100)
Top10: ['EFZ', 'CID', 'LVL', 'EUMV', 'IQDE']
Group: Developed Markets ETF List 3 (93)
Top10: ['FTRI', 'GDXS', 'FTAG', 'ROGS', 'GSD']
Group: Emerging Asia Pacific ETF List (114)
Top10: ['VWO', 'ADRE', 'CHAU', 'YIN

In [39]:
allgroup_top10_return.dropna(axis=0)

,Alternatives ETF List (35),Broad Asia ETF List (44),Commodity ETF List (125),Consumer Discretionary Equity ETF List (31),Consumer Staples Equity ETF List (26),Crude Oil ETF List (22),Currency ETF List (36),Developed Asia Pacific ETF List (108),Developed Markets ETF List 1 (100),Developed Markets ETF List 2 (100),...,Municipal Bond ETF List (29),Preferred Stock ETF List (12),Real Estate ETF List (48),Target Maturity Date Corporate Bond ETF List (24),Technology Equity ETF List (75),Telecom Equity ETF List (10),Total Bond Market ETF List (82),Treasuries ETF List (51),Utilities Equity ETF List (24),Volatility ETF List (18)
0,-0.016044,-0.031962,-0.020235,-0.073335,-0.027729,0.052912,-0.020300,-0.030649,-0.009976,-0.009966,...,0.009973,0.005460,-0.034582,0.009735,-0.074886,-0.039371,0.002967,-0.004249,0.010130,0.116312
1,0.004357,-0.002044,0.023987,0.000003,0.005959,-0.011919,0.007628,-0.026776,0.003529,-0.003568,...,0.001722,-0.001179,0.012123,0.007092,-0.001607,0.054965,-0.001732,-0.006112,0.010632,-0.002320
2,0.015708,0.087063,0.025385,0.092011,0.060982,-0.042491,0.042294,0.062633,0.079215,0.050192,...,0.002470,0.021737,0.073152,0.025161,0.091775,0.067985,0.003433,0.000770,0.069880,-0.155186
3,-0.006278,0.001224,0.070906,-0.002171,-0.000081,-0.063826,0.002909,0.029481,0.042018,0.015439,...,0.004885,0.012441,0.023711,0.011362,-0.030359,0.017243,0.002742,0.001721,-0.006333,-0.066510
4,-0.004883,-0.017423,-0.022358,0.011503,0.002041,-0.007363,-0.028051,-0.010974,-0.018422,-0.009580,...,0.001864,0.014598,0.000867,-0.000051,0.080506,-0.004888,0.002340,-0.003319,0.005120,-0.129731
5,0.002435,0.027172,0.054824,-0.012971,0.034285,-0.019166,0.013152,-0.019616,0.010804,-0.014509,...,0.010814,0.010515,0.003981,0.021768,-0.029310,0.028484,0.001615,-0.008262,0.031510,-0.095257
6,0.012787,0.055852,-0.024801,0.074987,0.031496,0.093026,0.005337,0.049060,0.038857,0.033290,...,-0.000351,0.019113,0.053170,0.012165,0.107443,0.048999,0.002424,-0.005199,0.020438,-0.156216
7,-0.006304,0.008553,-0.016152,-0.022311,-0.013586,-0.036678,-0.001136,-0.006682,-0.023202,-0.007226,...,0.000093,0.003497,-0.013129,-0.002593,0.044964,-0.014789,0.001985,0.000181,-0.037364,-0.240541
8,0.006187,0.016815,0.026782,-0.004666,-0.012393,-0.036376,0.008809,0.016892,0.017191,0.003737,...,-0.003318,-0.010645,-0.007394,-0.001011,0.025946,0.021056,0.001894,0.003469,0.003022,16.435751
9,-0.010920,-0.011434,0.004779,-0.035507,-0.022821,0.002634,-0.011448,-0.018176,-0.015640,-0.015153,...,-0.005133,-0.006732,-0.037289,-0.011028,-0.006155,-0.018429,0.000630,0.008731,-0.004317,-0.001400


# 將allgroup_top5 先存取成CSV

In [47]:
allgroup_top10_return.to_csv("allgroup_top5_return.csv")

# 再次計算Riskiness R，並依照R值排序，傳回前五名的Group

In [63]:
def get_top5_Group(data_file):

    df1 = pd.read_csv(data_file)
    etf_list1 = list(df1.columns)[1:]
    df1 = df1.iloc[:,1:]

    etf_dic1 = {}
    for etf_name in etf_list1:
        # 過濾平均值正負
        #if df[etf_name].mean(axis=0) < 0:
        #    continue
        arr_returns1 = df1[etf_name] - rfrate
        #arr_returns = arr_returns*1000      #調整數量級
        guess1 = 10**(-5)
        #### 列出細項
        #print('%s指標值:  %f' %(etf_name,risk2[0]) )
        etf_dic1[etf_name] = get_riskiness_r(guess1,arr_returns1)
    
    
    sorted_x1 = sorted(etf_dic1.items(), key=lambda kv: kv[1])
        #sorted_reversed_x = list(reversed(sorted_x))
    
    # 取前5名
    top5 = sorted_x1[:5]
        # top5 = sorted_reversed_x[:5]
    #print(top5)
    
    # 取前5的ETF名稱
    etf_top5 = [etf[0] for etf in top5]

    df2 = df1[etf_top5]
    
    return df2

In [20]:
def risk(x):
    guess  = 10**(-5)
    while (guess<1000):
        risk2 = fsolve(f1,guess,x)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]
    #return np.std(x)

def sumrisk(weight,all_return):
    w = weight
    a = all_return
    v = 0
    for i in range(len(w)):
        v = v + w[i]*risk(w[i]*(all_return.iloc[:,i]))
    return v

def penalty(w):
    if min(w)<0 or max(w)>1:
        return 100000
    else:
        return 0
def portretmean(weight,all_return):
    w = weight
    a = all_return
    return np.mean(np.dot(np.array(all_return),np.array(weight)))    

def bestweight(w):
    r = []
    r.append(penalty(w)+np.abs(sum(w)-1))
    for i in range(len(w)-1):
        v =  np.abs( risk(w[i]*(all_return.iloc[:,i]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i])-rfrate)/(portretmean(w,all_return)-rfrate) ) \
            + np.abs( risk(w[i+1]*(all_return.iloc[:,i+1]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i+1])-rfrate)/(portretmean(w,all_return)-rfrate))
        r.append(v) 
    print('權重:',w,'誤差值:',v+penalty(w)+np.abs(sum(w)-1))
    return r

In [21]:
def fourdrop(x):
    a = []
    for i in range(len(x)):
        a.append(round(sum(x[i]),3))
    return a
x = np.array(allgroup_top10_return.iloc[:,[0]])
x = fourdrop(x)
y = np.array(allgroup_top10_return.iloc[:,[1]])
y = fourdrop(y)
all_return = pd.DataFrame([x,y]).T
all_return = all_return

In [22]:
x = np.array([-0.01,0,0.0258])
y = np.array([-0.05,0,0.134])
all_return = pd.DataFrame([x,y]).T

# 依次計算32組的權重

In [33]:
a = list(range(0, 32))
all_return = allgroup_top10_return.iloc[:,a]
n = all_return.shape[1]
#weight = [0.4,0.6]
weight = [1/n]*n
r2 = fsolve(bestweight, weight)
print(r2)

C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)
C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738875873
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738875873
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738875873
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125

權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.74938272427682
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.74938273561566
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738560733
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0

權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382737570457
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.74938275550922
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738476218
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 

# 計算出group中 前五名的權重

In [64]:
df2 = get_top5_Group("C:\\Users\\USER\\HW1--data-mining\\FinalProject\\Scripts\\allgroup_top5_return.csv")

C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


In [65]:
a = list(range(0, 5))
all_return = df2.iloc[:,a]
n = all_return.shape[1]
#weight = [0.4,0.6]
weight = [1/n]*n
r2 = fsolve(bestweight, weight)
print(r2)

權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699311450244
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699311450244
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699311450244
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699277772157
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699336919056
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699334164806
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699047231302
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482698906390747
權重: [-2.33841759  1.32100946  1.05216177  0.46625234  0.49899401] 誤差值: 100000.91648598296
權重: [-0.42525488 -0.35552826 -0.16480408  0.32225977  0.35361388] 誤差值: 100127.83470031185
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699277772157
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699336919056
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699334164806
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699047231302
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482698906390747
權重: [-2.33841759  1.32100946  1.05216177  0.46625234  0.49899401] 誤差值: 100000.91648598296
權重: [-0.42525488 -0.35552826 -0.16480408  0.32225977  0.35361388] 誤差值: 100127.83

權重: [0.14213777 0.05990256 0.12145915 0.39664978 0.37059719] 誤差值: 0.6059275484659522
權重: [0.14212823 0.05989717 0.1213798  0.39664708 0.37061379] 誤差值: 0.6058094049762985
權重: [0.14212187 0.05989358 0.12132691 0.39664529 0.37062485] 誤差值: 0.6057306457460933
權重: [0.14210918 0.05988643 0.12122113 0.39664169 0.37064699] 誤差值: 0.5920312623363115
權重: [0.14212822 0.05989716 0.1213798  0.39664709 0.37061378] 誤差值: 0.6058094061051824
權重: [0.14212187 0.05989358 0.12132691 0.39664529 0.37062485] 誤差值: 0.6057306473835821
權重: [0.14212187 0.05989359 0.12132691 0.39664529 0.37062485] 誤差值: 0.6057306460799412
權重: [0.14212187 0.05989358 0.12132691 0.39664529 0.37062485] 誤差值: 0.6057306467099904
權重: [0.14212187 0.05989358 0.12132691 0.39664529 0.37062485] 誤差值: 0.6057306651414192
權重: [0.14212187 0.05989358 0.12132691 0.39664529 0.37062486] 誤差值: 0.6057306265810619
權重: [0.1421187  0.05989179 0.12130046 0.39664438 0.37063038] 誤差值: 0.5921370041775393
[0.14212187 0.05989358 0.12132691 0.39664529 0.37062485]


C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
